In [1]:
import pandas as pd

In [2]:
london_data = pd.read_csv('london_data.csv')
london_data.head()

/var/folders/nc/3_jdz57s3fb248nc5jbmskj40000gn/T/ipykernel_67861/2557532795.py:1: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  london_data = pd.read_csv('london_data.csv')


,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
0,134489498,2023-09-30 23:59,1083,"Commercial Street, Shoreditch",2023-10-01 00:08,200121,"Shoreditch Court, Haggerston",57640,CLASSIC,9m 1s,541565
1,134489510,2023-09-30 23:59,3504,"Moor Street, Soho",2023-10-01 01:02,300234,"Stratford Station, Stratford",57335,CLASSIC,1h 2m 59s,3779551
2,134489509,2023-09-30 23:59,3504,"Moor Street, Soho",2023-10-01 01:04,300234,"Stratford Station, Stratford",30035,CLASSIC,1h 4m 27s,3867026
3,134489508,2023-10-01 00:00,1133,"Baylis Road, Waterloo",2023-10-01 00:06,1078,"Lambeth Road, Vauxhall",52136,CLASSIC,5m 59s,359570
4,134489507,2023-09-30 23:59,300079,"London Street, Paddington",2023-10-01 00:16,1214,"Kensington Olympia Station, Olympia",57534,CLASSIC,16m 29s,989750


In [3]:
london_data['End date']

0          2023-10-01 00:08
1          2023-10-01 01:02
2          2023-10-01 01:04
3          2023-10-01 00:06
4          2023-10-01 00:16
                 ...       
8090408    2023-05-29 00:03
8090409    2023-05-29 00:14
8090410    2023-05-29 00:03
8090411    2023-05-29 00:04
8090412    2023-05-29 00:24
Name: End date, Length: 8090413, dtype: object

In [4]:
london_data.shape

(8090413, 11)

In [ ]:
dublin_data = pd.read_csv('dublin_data_combined.csv')
dublin_data.head()

In [ ]:
dublin_data.shape

Getting Bike station Locations for London and Dublin

In [ ]:
# Step 1: Create a copy of the original DataFrame
london_data_copy = london_data.copy()

# Step 2: Clean the copied DataFrame

# Drop rows with missing values in specified columns
london_data_copy.dropna(axis=0, subset=["Start station", "End station", "Start date", "End date"], inplace=True)


# Filter out rows where StartStation Id is equal to EndStation Id
london_data_copy = london_data_copy[london_data_copy["Start station"] != london_data_copy["End station"]]

# Keep only the specified columns
london_data_copy = london_data_copy.loc[:, ['Start date', 'Start station', 'End date', 'End station', 'Total duration']]

# Drop duplicates
london_data_copy.drop_duplicates(inplace=True)

# Now you can work with london_data_copy without affecting the original london_data
print(london_data_copy.shape)
london_data_copy.head()

In [ ]:
import requests
from xml.etree import ElementTree as ET

In [ ]:
site = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

response = requests.get(site)
root = ET.fromstring(response.content)

id_list = [int(root[i][0].text) for i in range(0, len(root))]
name_list = [root[i][1].text for i in range(0, len(root))]
lat_list = [float(root[i][3].text) for i in range(0, len(root))]
lon_list = [float(root[i][4].text) for i in range(0, len(root))]
capacity_list = [int(root[i][12].text) for i in range(0, len(root))]

london_all_locs = pd.DataFrame(list(zip(name_list, id_list, lat_list, 
                                 lon_list, capacity_list)), columns = ["name","id","lat","lon","capacity"])

london_all_locs.to_csv('bike_point_locations_saved.csv', header=True, index=None)

print(london_all_locs.shape)
london_all_locs.head()

In [ ]:
london_data['Bike number'].unique()

In [ ]:
print('London Bikes : ' ,len(london_data['Bike number'].unique()))
print('London Stations : ' ,london_all_locs.shape[0])

In [ ]:
dublin_all_locs = dublin_data[['STATION ID','NAME','BIKE_STANDS','STATUS','LATITUDE', 'LONGITUDE']].drop_duplicates(subset=["STATION ID"])
dublin_all_locs.head()

In [ ]:
print('Dublin Bikes : ' ,dublin_all_locs['BIKE_STANDS'].sum())
print('Dublin Stations : ' ,dublin_all_locs.shape[0])